In [1]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
import pandas as pd

In [2]:
df = pd.read_csv('spam.csv',encoding= 'ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.drop(columns= ['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace= True)

In [4]:
df.rename(columns = {'v1':'TargetVar','v2':'Text'},inplace = True)

In [5]:
df.isnull().sum()

TargetVar    0
Text         0
dtype: int64

In [6]:
df.duplicated().sum()

403

In [7]:
df.drop_duplicates(inplace= True)

In [8]:
df['TargetVar'] = encoder.fit_transform(df['TargetVar'])

In [9]:
df.head()

,TargetVar,Text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
def TextCleaner(string):
    string = string.lower()      # 1)
    tokenized_Str = nltk.word_tokenize(string)  # 2)
    cleaned_text = []
    for item in tokenized_Str :
        if item.isalnum():
            if item not in stopwords.words('english'):
                cleaned_text.append(item)
    stemmed_text = []
    for word in cleaned_text:
        stemmed_text.append(ps.stem(word))
        
    return " ".join(stemmed_text)

In [11]:
df['Cleaned_Text'] = df['Text'].apply(TextCleaner)

In [12]:
df.head()

,TargetVar,Text,Cleaned_Text
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(max_features=3000)

In [15]:
X = df['Cleaned_Text']

In [16]:
Y = df['TargetVar']

In [17]:
from sklearn.model_selection import train_test_split as tts

In [18]:
x_train,x_test,y_train,y_test = tts(X,Y,test_size= 0.15)

In [19]:
x_train = vectorizer.fit_transform(x_train).toarray()
x_train

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.3390217],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [20]:
x_test = vectorizer.transform(x_test)
x_test

<776x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 5345 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.naive_bayes import MultinomialNB

In [22]:
model = MultinomialNB()

In [23]:
model.fit(x_train,y_train)

MultinomialNB()

In [24]:
model.predict(x_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,

In [25]:
cleantext = TextCleaner('My name is noorain. you have won 5000dollars')
cleantext

'name noorain 5000dollar'

In [26]:
veccleantext = vectorizer.transform([cleantext])
veccleantext

<1x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [27]:
model.predict(veccleantext)

array([0])

In [28]:
import pickle 

In [29]:
pickle.dump(model,open("model.pkl",'wb'))

In [30]:
pickle.dump(vectorizer,open("vectorizer.pkl",'wb'))